<a href="https://colab.research.google.com/github/pra17dod/Waste-Segregation/blob/main/model/v3_waste_segregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import re
import random
import seaborn as sns
import cv2 as cv
from PIL import Image
import pickle
import sys
sys.setrecursionlimit(1000000000)

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing import image_dataset_from_directory
# from tfhub import load_model_weights

In [ ]:
!unzip /content/drive/MyDrive/Hackathon/dataset-resized.zip -d /content/

Archive:  /content/drive/MyDrive/Hackathon/dataset-resized.zip
replace /content/dataset-resized/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/dataset-resized/.DS_Store  
replace /content/dataset-resized/cardboard/cardboard1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/dataset-resized/cardboard/cardboard1.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard10.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard100.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard101.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard102.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard103.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard104.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard105.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard106.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard107.jpg  
  inflating: /content/dataset-resized

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.listdir(os.path.join(os.getcwd(),"dataset-resized"))

['metal',
 '.DS_Store',
 'organic',
 'plastic',
 'paper',
 'glass',
 'trash',
 'cardboard']

In [ ]:
## get a path to the folder with images
path = os.path.join(os.getcwd(),"dataset-resized")
path

'/content/dataset-resized'

In [ ]:
print(len(os.listdir('./dataset-resized//')))

8


In [ ]:
train = tf.keras.preprocessing.image_dataset_from_directory(path, batch_size=16, image_size=(224,224), shuffle=True, label_mode='categorical', validation_split = 0.25, seed = 1, subset = 'training')
valid = tf.keras.preprocessing.image_dataset_from_directory(path, batch_size=16, image_size=(224,224), shuffle=True, label_mode='categorical', validation_split = 0.25, seed = 1, subset = 'validation')

Found 3928 files belonging to 7 classes.
Using 2946 files for training.
Found 3928 files belonging to 7 classes.
Using 982 files for validation.


In [ ]:
model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224,224,3))

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
def waste_segment():
    model2 = tf.keras.Sequential()
    model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224,224,3), weights='imagenet')
    model2.add(model)
    model2.add(tf.keras.layers.Flatten())
    model2.add(tf.keras.layers.Dense(6, activation='softmax'))
    return model2

In [ ]:
model2 = tf.keras.Sequential()
model2.add(model)
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(Dense(512, activation = 'relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(Dense(256, activation = 'relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(Dense(128, activation = 'relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(Dense(64, activation = 'relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(7, activation='softmax'))

In [ ]:
# model2.load_weights('/content/drive/MyDrive/waste segmentation/waste_weights.h5')

In [ ]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 100352)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               51380736  
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)              

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.Adam(), loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])

In [ ]:
model2.fit(x=train, validation_data=valid, epochs = 10)

Epoch 1/10
185/185 [==============================] - 27s 147ms/step - loss: 0.2082 - accuracy: 0.9477 - val_loss: 1.0266 - val_accuracy: 0.7363
Epoch 2/10
185/185 [==============================] - 27s 147ms/step - loss: 0.1929 - accuracy: 0.9487 - val_loss: 1.0046 - val_accuracy: 0.7525
Epoch 3/10
185/185 [==============================] - 27s 146ms/step - loss: 0.1531 - accuracy: 0.9559 - val_loss: 0.9977 - val_accuracy: 0.7587
Epoch 4/10
185/185 [==============================] - 27s 146ms/step - loss: 0.2277 - accuracy: 0.9396 - val_loss: 1.6349 - val_accuracy: 0.6344
Epoch 5/10
185/185 [==============================] - 27s 145ms/step - loss: 0.1989 - accuracy: 0.9443 - val_loss: 1.1497 - val_accuracy: 0.7240
Epoch 6/10
185/185 [==============================] - 27s 145ms/step - loss: 0.1903 - accuracy: 0.9525 - val_loss: 1.2766 - val_accuracy: 0.7251
Epoch 7/10
185/185 [==============================] - 27s 146ms/step - loss: 0.2201 - accuracy: 0.9569 - val_loss: 1.2270 - val_ac

In [ ]:
while(1):
  pass

In [ ]:
temp = img_to_array(load_img('/content/dataset-resized/cardboard/cardboard100.jpg', target_size=(224,224))).reshape(1,224,224,3)
np.argmax(model2.predict(temp))


0

In [ ]:
model2.save_weights('waste_weights_heavy.h5')

In [ ]:
!cp waste_weights_heavy.h5 /content/drive/MyDrive/Hackathon/waste_weights_heavy.h5